In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import tensorflow


In [3]:
base_dir = "../input/dogs-vs-cats/"
train_dir = os.path.join(base_dir, "train.zip")
test_dir = os.path.join(base_dir, "test1.zip")

import zipfile
with zipfile.ZipFile(train_dir,"r") as z:
    z.extractall()

with zipfile.ZipFile(test_dir,"r") as z:
    z.extractall()

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
train_data = ImageDataGenerator(rescale = 1./255)


In [5]:
!mkdir train/dog
!mkdir train/cat

In [8]:
import shutil
dog = "/kaggle/working/train/dog"
cat = "/kaggle/working/train/cat"
path = "/kaggle/working/train"
for i in os.listdir("/kaggle/working/train"):
    if "cat" in i:
        shutil.move("/kaggle/working/train/"+i, cat)
    else:
        shutil.move("/kaggle/working/train/"+i, dog)
        
 


In [9]:
cd ../

In [10]:
!mkdir validation

In [11]:
!mkdir validation/dog
!mkdir validation/cat

In [ ]:
### Validation Directory

In [12]:
cat_train = "/kaggle/working/train/cat"
dog_train = "/kaggle/working/train/dog"


In [13]:
validation_cat = '/kaggle/working/validation/cat/'

In [14]:
validation_dog = '/kaggle/working/validation/dog/'

In [19]:
co = 0 
for i in os.listdir(cat_train):
    if co < 2000:
        shutil.move(cat_train+"/"+i, validation_cat)
    else:
        break
    co = co+1
    


In [20]:
co = 0 
for i in os.listdir(dog_train):
    if co < 2000:
        shutil.move(dog_train+"/"+i, validation_dog)
    else:
        break
    co = co+1
    


In [18]:
import tensorflow
from tensorflow.keras.preprocessing.image import ImageDataGenerator
#from tensorflow.keras.preprocessing.Image import ImageDataGenerator

In [21]:
train_data = ImageDataGenerator(1./255)
train_gen = train_data.flow_from_directory(
    '/kaggle/working/train/',
    target_size = (150,150),
    batch_size = 20,
    class_mode = 'binary'
     )

In [22]:
validation_gen = ImageDataGenerator(rescale = 1./255)
validation_data = validation_gen.flow_from_directory('/kaggle/working/validation/',
    target_size = (150,150),
    batch_size = 20,
    class_mode = 'binary'
)

In [23]:
import tensorflow as tf

In [24]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(16,(3,3),activation="relu",input_shape=(150,150,3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32,(3,3),activation="relu"),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32,(3,3),activation="relu"),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512,activation="relu"),
    tf.keras.layers.Dense(1,activation="sigmoid")
])

In [25]:
model.summary()

In [26]:
from tensorflow.keras.optimizers import RMSprop
model.compile(optimizer = RMSprop(learning_rate=0.001),
             loss='binary_crossentropy',
             metrics=['accuracy'])

In [27]:
history = model.fit(train_gen,validation_data = validation_data,steps_per_epoch = 500, epochs = 20, validation_steps = 50,verbose = 2)

In [ ]:
len(os.listdir(cat))
len(os.listdir(dog))

In [28]:
import numpy as np

from google.colab import files
from keras.preprocessing import image
uploaded = files.upload()

for fn in uploaded.keys():
    
    path = '/content/' + fn;
    img=image.load_img(path, target_size=(150, 150))
    x=image.img_to_array(img)
    x=np.expand_dims(x, axis=0)
    images = np.vstack([x])
    classes = model.predict(images, batch_size=10)
  
    print(classes[0])
  
    if classes[0]>0:
        print(fn + " is a dog")
    else:
        print(fn + " is a cat")
        


In [32]:
import matplotlib.pyplot as plt
#-----------------------------------------------------------
# Retrieve a list of list results on training and test data
# sets for each training epoch
#-----------------------------------------------------------
acc      = history.history[     'accuracy' ]
val_acc  = history.history[ 'val_accuracy' ]
loss     = history.history[    'loss' ]
val_loss = history.history['val_loss' ]

epochs   = range(len(acc)) # Get number of epochs

#------------------------------------------------
# Plot training and validation accuracy per epoch
#------------------------------------------------
plt.plot  ( epochs,     acc )
plt.plot  ( epochs, val_acc )
plt.title ('Training and validation accuracy')
plt.figure()

#------------------------------------------------
# Plot training and validation loss per epoch
#------------------------------------------------
plt.plot  ( epochs,     loss )
plt.plot  ( epochs, val_loss )
plt.title ('Training and validation loss'   )